In [1]:
# % matplotlib tk
import matplotlib.pyplot as plt
import numpy as np

from PIL import Image

In [2]:
def get_input_lines(im, min_lines=3):
    """
    Allows user to input line segments; computes centers and directions.
    Inputs:
        im: np.ndarray of shape (height, width, 3)
        min_lines: minimum number of lines required
    Returns:
        n: number of lines from input
        lines: np.ndarray of shape (3, n)
            where each column denotes the parameters of the line equation
        centers: np.ndarray of shape (3, n)
            where each column denotes the homogeneous coordinates of the centers
    """
    n = 0
    lines = np.zeros((3, 0))
    centers = np.zeros((3, 0))

    plt.figure()
    plt.imshow(im)
    print('Set at least %d lines to compute vanishing point' % min_lines)
    while True:
        print('Click the two endpoints, use the right key to undo, and use the middle key to stop input')
        clicked = plt.ginput(2, timeout=0, show_clicks=True)
        if not clicked or len(clicked) < 2:
            if n < min_lines:
                print('Need at least %d lines, you have %d now' % (min_lines, n))
                continue
            else:
                # Stop getting lines if number of lines is enough
                break

        # Unpack user inputs and save as homogeneous coordinates
        pt1 = np.array([clicked[0][0], clicked[0][1], 1])
        pt2 = np.array([clicked[1][0], clicked[1][1], 1])
        # Get line equation using cross product
        # Line equation: line[0] * x + line[1] * y + line[2] = 0
        line = np.cross(pt1, pt2)
        lines = np.append(lines, line.reshape((3, 1)), axis=1)
        # Get center coordinate of the line segment
        center = (pt1 + pt2) / 2
        centers = np.append(centers, center.reshape((3, 1)), axis=1)

        # Plot line segment
        plt.plot([pt1[0], pt2[0]], [pt1[1], pt2[1]], color='b')

        n += 1

    return n, lines, centers

In [3]:
def plot_lines_and_vp(im, lines, vp):
    """
    Plots user-input lines and the calculated vanishing point.
    Inputs:
        im: np.ndarray of shape (height, width, 3)
        lines: np.ndarray of shape (3, n)
            where each column denotes the parameters of the line equation
        vp: np.ndarray of shape (3, )
    """
    bx1 = min(1, vp[0] / vp[2]) - 10
    bx2 = max(im.shape[1], vp[0] / vp[2]) + 10
    by1 = min(1, vp[1] / vp[2]) - 10
    by2 = max(im.shape[0], vp[1] / vp[2]) + 10

    plt.figure()
    plt.imshow(im)
    for i in range(lines.shape[1]):
        if lines[0, i] < lines[1, i]:
            pt1 = np.cross(np.array([1, 0, -bx1]), lines[:, i])
            pt2 = np.cross(np.array([1, 0, -bx2]), lines[:, i])
        else:
            pt1 = np.cross(np.array([0, 1, -by1]), lines[:, i])
            pt2 = np.cross(np.array([0, 1, -by2]), lines[:, i])
        pt1 = pt1 / pt1[2]
        pt2 = pt2 / pt2[2]
        plt.plot([pt1[0], pt2[0]], [pt1[1], pt2[1]], 'g')

    plt.plot(vp[0] / vp[2], vp[1] / vp[2], 'ro')
    plt.show()

In [4]:
def get_top_and_bottom_coordinates(im, obj):
    """
    For a specific object, prompts user to record the top coordinate and the bottom coordinate in the image.
    Inputs:
        im: np.ndarray of shape (height, width, 3)
        obj: string, object name
    Returns:
        coord: np.ndarray of shape (3, 2)
            where coord[:, 0] is the homogeneous coordinate of the top of the object and coord[:, 1] is the homogeneous
            coordinate of the bottom
    """
    plt.figure()
    plt.imshow(im)

    print('Click on the top coordinate of %s' % obj)
    clicked = plt.ginput(1, timeout=0, show_clicks=True)
    x1, y1 = clicked[0]
    # Uncomment this line to enable a vertical line to help align the two coordinates
    # plt.plot([x1, x1], [0, im.shape[0]], 'b')
    print('Click on the bottom coordinate of %s' % obj)
    clicked = plt.ginput(1, timeout=0, show_clicks=True)
    x2, y2 = clicked[0]

    plt.plot([x1, x2], [y1, y2], 'b')

    return np.array([[x1, x2], [y1, y2], [1, 1]])

In [5]:

"""
Solves for the vanishing point using the user-input lines.
"""
def get_vanishing_point(im,lines):
    h,w = im.shape[0],im.shape[1]
    vp = [0] * 3
    left_tmp = np.transpose( lines[:2,:] )
    right_tmp = np.transpose(-lines[2, :])
    vp[0],vp[1] = np.linalg.lstsq(left_tmp,right_tmp)[0]
    vp[2] = 1
    return vp

In [6]:

import matplotlib.lines as mlines
def get_horizon_line(vpts):
    """
    Calculates the ground horizon line.
    """
    horizon = np.real(np.cross(
        np.transpose(vpts[0,:]),
        np.transpose(vpts[1,:])
    ))
    scale_factor = np.sqrt(horizon[0]**2+horizon[1]**2)
    return np.true_divide(horizon,scale_factor)

def plot_horizon_line(im, hline):
    print("horizontal line:",hline)
    print(im.shape)
    plt.figure()
    plt.imshow(im)
    x1 = 0
    y1 = - hline[2]/hline[1]
    x2 = im.shape[1]
    y2 =  -hline[0]/hline[1]* x2 - hline[2]/hline[1]
    print([x1,y1 ], [x2, y2])
    
    plt.plot([x1, x2],[y1,y2],  'r')
    plt.show()

In [12]:
import sympy
def get_camera_parameters(vpts):
    
    # its vp_z, vp_x, vp_y
    vp_1 = np.transpose( vpts[0,:] )
    vp_2 = np.transpose( vpts[1,:] )
    vp_3 = np.transpose( vpts[2,:] )
    
    vp_z,vp_x,vp_y = vp_1,vp_2,vp_3
    
    print("vp_X:",vp_1)
    print("vp_Y:",vp_2)
    print("vp_z:",vp_3)
    u = sympy.Symbol('u')
    v = sympy.Symbol('v')
    f = sympy.Symbol('f')
    sol = sympy.solve([
        -u*(vp_2[0]+vp_3[0]-vp_3[0]-vp_1[0]) 
        + vp_2[0]*vp_3[0]-vp_3[0]*vp_1[0]
        - v*(vp_2[1]+vp_3[1]-vp_3[1]-vp_1[1])
        + vp_2[1]*vp_3[1]-vp_3[1]*vp_1[1]
        ,
         -u*(vp_1[0]+vp_2[0]-vp_3[0]-vp_1[0]) 
        + vp_1[0]*vp_2[0]-vp_3[0]*vp_1[0]
        - v*(vp_1[1]+vp_2[1]-vp_3[1]-vp_1[1])
        + vp_1[1]*vp_2[1]-vp_3[1]*vp_1[1]
    ],[u,v]
        
         )
    # sol = ( sympy.solve([-u*(vp_x[0]+vp_y[0]-vp_y[0]-vp_z[0]) +\
    #              vp_x[0]*vp_y[0]-vp_y[0]*vp_z[0] + \
    #              -v*(vp_x[1]+vp_y[1]-vp_y[1]-vp_z[1]) +\
    #              vp_x[1]*vp_y[1]-vp_y[1]*vp_z[1] 
    #             ,
    #               -u*(vp_z[0]+vp_x[0]-vp_y[0]-vp_z[0]) +\
    #              vp_z[0]*vp_x[0]- vp_y[0]*vp_z[0] + \
    #              -v*(vp_z[1]+vp_x[1]-vp_y[1]+vp_z[1]) +\
    #              vp_z[1]*vp_x[1]-vp_y[1]*vp_z[1] 
    #              ],[u,v]))
   
    u_sol,v_sol = sol[u],sol[v]
    u = sympy.Float(u_sol)
    v = sympy.Float(v_sol)
    f_sol = sympy.solve((u-vp_x[0])*(u-vp_y[0]) 
                        + (v-vp_x[1]) * (v - vp_y[1]) + f*f ,f )
    f = sympy.Float((f_sol[0]))
    return float(f),float(u),float(v)
    

In [13]:
def get_rotation_matrix(Calibration,vpts):
    """
    Computes the rotation matrix using the camera parameters.
    """
    vp_1 = np.transpose( vpts[0,:] )
    vp_2 = np.transpose( vpts[1,:] )
    vp_3 = np.transpose( vpts[2,:] )
    r_z = np.linalg.lstsq(Calibration, vp_1)[0]
    r_x = np.linalg.lstsq(Calibration, vp_2)[0]
    r_y = np.linalg.lstsq(Calibration, vp_3)[0]
    
    r_x = r_x/np.linalg.norm(r_x)
    r_y = r_y/np.linalg.norm(r_y)
    r_z = r_z/np.linalg.norm(r_z)
    
    return np.vstack((np.vstack((r_x,r_y)),r_z))

In [14]:
def estimate_height(im,target_coor,height_ref,ref_coor,horizon_line,vpts):
    """
    Estimates height for a specific object using the recorded coordinates. 
    You might need to plot additional images here for
    your report.
    """
    # its vp_z, vp_x, vp_y
    vp_1 = np.transpose( vpts[0,:] )
    vp_2 = np.transpose( vpts[1,:] )
    vp_3 = np.transpose( vpts[2,:] )
    # top and bottom point for referece
    rx1,rx2 = ref_coor[0]
    ry1,ry2 = ref_coor[1]
    #refrence top and bottom vector
    
    refer_bottom = np.array([rx1,ry1,1])
    refer_top = np.array([rx2,ry2,1])
    
    # top and bottom point for target
    tx1,tx2 = target_coor[0]
    ty1,ty2 = target_coor[1]
    tar_img_height = ty2 - ty1
    
    target_bottom = np.array([tx1,ty1,1])
    target_top = np.array([tx2,ty2,1])
    line_bottom = np.real(np.cross(np.transpose(refer_bottom),np.transpose(target_bottom)))
    V = np.real(np.cross(line_bottom, horizon_line))
    V = V/V[2]
    # print("V:",V)
    
    line_top = np.real(np.cross(np.transpose(V),np.transpose(refer_top)))
    
    line_ver = np.real(np.cross(np.transpose(target_top),np.transpose(target_bottom)))
    T = np.real(np.cross(np.transpose(line_top),np.transpose(line_ver)))
    T = T/T[2]
    
    plt.figure()
    plt.imshow(im)
    plt.plot([V[0],refer_bottom[0]],[V[1],refer_bottom[1]],  'r')
    plt.plot([V[0],refer_top[0]],[V[1],refer_top[1]],  'r')
    
    plt.plot([V[0],T[0]],[V[1],T[1]],  'r')
    plt.plot([tx1,tx2],[ty1,ty2], 'y')
    plt.plot([rx1,rx2],[ry1,ry2], 'y')
    plt.plot([target_bottom[0],T[0]],[target_bottom[1],T[1]],  'g+')
    plt.plot([target_bottom[0],V[0]],[target_bottom[1],V[1]],  'b+')

    # plt horizon line
    x1 = 0
    y1 = - horizon_line[2]/horizon_line[1]
    x2 = im.shape[1]
    y2 =  -horizon_line[0]/horizon_line[1]* x2 - horizon_line[2]/horizon_line[1]
    print([x1,y1 ], [x2, y2])
    
    plt.plot([x1, x2],[y1,y2],  'w')
    plt.show()
    
    up = np.linalg.norm(T-target_bottom) * np.linalg.norm(np.transpose(vp_3)-target_top)
    down = np.linalg.norm(target_top-target_bottom) * np.linalg.norm(np.transpose(vp_3)-T)
    return (height_ref * down) / up
    

In [15]:
CSL = "data\\part3\\CSL.jpg"
im = np.asarray(Image.open(CSL))

# Part 1
# Get vanishing points for each of the directions

# num_vpts = 3
# vpts = np.zeros((3, num_vpts))
# for i in range(num_vpts):
#     print('Getting vanishing point %d' % i)
#     # Get at least three lines from user input
#     n, lines, centers = get_input_lines(im)
#     # <YOUR IMPLEMENTATION> Solve for vanishing point
#     vpts[:, i] = get_vanishing_point(im,lines)
#     print(vpts)
#     # Plot the lines and the vanishing point
#     plot_lines_and_vp(im, lines, vpts[:, i])
# 
# print(vpts)

# results after many times average 
vpts = np.array([
    [-202, 215 ,1],
    [1371 ,230 ,1],
    [503 ,4867, 1]
])
# plot horizon line
horizon_line = get_horizon_line(vpts)
# plot_horizon_line(im, horizon_line)

# Part 2
# <YOUR IMPLEMENTATION> Solve for the camera parameters (f, u, v)

f, u, v = get_camera_parameters(vpts)
f = abs(f)
print("f:",f)
print("u:",u)
print("v:",v)
# 
# # Part 3
# <YOUR IMPLEMENTATION> Solve for the rotation matrix
Calibration_Matrix = np.array( [
    [f, 0, u],
    [0, f, v],
     [0, 0, 1]
    ]
)
R = get_rotation_matrix(Calibration_Matrix,vpts)
print("rota M:",R)
# 
# # Part 4
# # Record image coordinates for each object and store in map


objects = ('person', 'CSL building', 'the spike statue', 'the lamp posts')

# coords = dict()
# for obj in objects:
#     coords[obj] = get_top_and_bottom_coordinates(im, obj)
# 
# print("coords:",coords)

# record result so test will be faster
coords = {'person': np.array([[625.0483871 , 625.0483871 ],
       [467.10967742, 508.4       ],
       [  1.        ,   1.        ]]), 'CSL building': np.array([[474.33870968, 476.40322581],
       [153.30322581, 304.01290323],
       [  1.        ,   1.        ]]), 'the spike statue': np.array([[605.27419355, 605.27419355],
       [198.30322581, 471.23870968],
       [  1.        ,   1.        ]]), 'the lamp posts': np.array([[709.69354839, 707.62903226],
       [326.72258065, 401.04516129],
       [  1.        ,   1.        ]])}

# # <YOUR IMPLEMENTATION> Estimate heights


# 5 f 6 inch = 167.64cm
# 6 f = 182.88cm
height_ref = 167.64


for obj in objects[1:]:
    print('Estimating height of %s' % obj)
    height = estimate_height(im,coords[obj],height_ref,coords[objects[0]],horizon_line,vpts)
    print("Its height is:",height)

vp_X: [-202  215    1]
vp_Y: [1371  230    1]
vp_z: [ 503 4867    1]


f: 774.3357839650271
u: 546.0258459637655
v: 355.0229532664543
rota M: [[ 0.72471933 -0.10982956  0.68023478]
 [-0.0093981   0.98554764  0.16913756]
 [-0.68898009 -0.12897018  0.71321324]]
Estimating height of CSL building
[0, 216.9262555626192] [1024, 226.69103623649076]


C:\Users\bluerain\PycharmProjects\cvmp2\new_venv\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  
C:\Users\bluerain\PycharmProjects\cvmp2\new_venv\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  if __name__ == '__main__':
C:\Users\bluerain\PycharmProjects\cvmp2\new_venv\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the inpu

Its height is: 2195.360883016264
Estimating height of the spike statue
[0, 216.9262555626192] [1024, 226.69103623649076]


Its height is: 11792.581865890435
Estimating height of the lamp posts
[0, 216.9262555626192] [1024, 226.69103623649076]


Its height is: 685.1921096214987
